In [1]:
import sys
import numpy as np
from matplotlib import pyplot
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from icecream import ic
import pandas as pd

import random

ic("USING pytorch VERSION: ", torch.__version__)

ic| 'USING pytorch VERSION: ', torch.__version__: '1.6.0'


('USING pytorch VERSION: ', '1.6.0')

## Define a pytorch Dataset object to contain the training and testing data
Pytorch handles data shuffling and batch loading, as long as the user provides a "Dataset" class. This class is just a wrapper for your data that casts the data into pytorch tensor format and returns slices of the data. In this case, our data is in numpy format, which conveniently pytorch has a method for converting to their native format.

The init function takes the path to the csv and creates a dataset out of it. I actually have three different options here. The dataset could be composed such that x is the 'timestamp' of the movement,the previous set of angles, or a tuple of both.

In [57]:
class AngleDataset(Dataset):
    def __init__(self, x, y):
        x_dtype = torch.FloatTensor
        y_dtype = torch.FloatTensor     # for MSE or L1 Loss

        self.length = x.shape[0]

        self.x_data = torch.from_numpy(x).type(x_dtype)
        self.y_data = torch.from_numpy(y).type(y_dtype)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.length


def create_datasets(csv_path: str, train_perc: float = 0.8):
    df = pd.read_csv(csv_path)
    length = len(df)
    time = 10
    timestep = 0.005

    # x_data = np.array([])
    # y_data = np.array([])
    x_data = []
    y_data = []

    sin_test_timepoints = np.random.rand(length, 1)*time   # Repeat data generation for test set
    sin_test_timepoints = sin_test_timepoints.ravel()
    sin_iter = iter(sin_test_timepoints)
    
    #data order = sin, angles, torso, touch_sens

    # if x = curr angles and y = next angles
    for i in range(len(df)):

        x = []
        y = []

        if i < length - 1:
            x = np.append(x, df.iloc[i])
            y = np.append(y, df.iloc[i+1][:-4]) # only include angles
        else:
            #since it loops anyway
            x = np.append(x, df.iloc[i])
            y = np.append(y, df.iloc[0][:-4])

        x = np.append([next(sin_iter)], x)
        
        
        x_data.append(x)
        y_data.append(y)
    
  
    x_data = np.array(x_data, dtype=np.float64)
    y_data = np.array(y_data, dtype=np.float64)

    last_train_idx = int(len(x_data) * train_perc)

    train_x = x_data[:last_train_idx]
    train_y = y_data[:last_train_idx]
    test_x = x_data[last_train_idx:]
    test_y = y_data[last_train_idx:]

    return AngleDataset(x=train_x, y=train_y), AngleDataset(x=test_x, y=test_y)

train_dataset, test_dataset = create_datasets('./walk_angles.csv')

train_dataset.x_data.shape

        


torch.Size([75, 33])

## Define training methods for the model
These methods use an initialized model and training data to iteratively perform the forward and backward pass of optimization. Aside from some data reformatting that depends on the input, output, and loss function, these methods will always be the same for any shallow neural network.

In [58]:
def train_batch(model, x, y, optimizer, loss_fn):
    # Run forward calculation
    y_predict = model.forward(x)

    # Compute loss.
    loss = loss_fn(y_predict, y)

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

    return loss.data.item()


def train(model, loader, optimizer, loss_fn, epochs=5):
    losses = list()

    batch_index = 0
    for e in range(epochs):
        for x, y in loader:
            loss = train_batch(model=model, x=x, y=y, optimizer=optimizer, loss_fn=loss_fn)
            losses.append(loss)

            batch_index += 1

        if e % 50 == 0:
          ic("Epoch: ", e+1)
          ic("Batches: ", batch_index)

    return losses


## Define testing methods for the model
These methods are like training, but we don't need to update the parameters of the model anymore because when we call the test() method, the model has already been trained. Instead, this method just calculates the predicted y values and returns them, AKA the forward pass.


In [59]:
def test_batch(model, x, y):
    # run forward calculation
    y_predict = model.forward(x)

    return y, y_predict


def test(model, loader):
    y_vectors = list()
    y_predict_vectors = list()

    batch_index = 0
    for x, y in loader:
        y, y_predict = test_batch(model=model, x=x, y=y)

        y_vectors.append(y.data.numpy())
        y_predict_vectors.append(y_predict.data.numpy())

        batch_index += 1

    y_predict_vector = np.concatenate(y_predict_vectors)

    return y_predict_vector


## Define plotting method for loss
This is a plotting method for looking at the behavior of the loss over training iterations.

In [60]:
def plot_loss(losses, show=True):
    fig = pyplot.gcf()
    fig.set_size_inches(8,6)
    ax = pyplot.axes()
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Loss")
    x_loss = list(range(len(losses)))
    pyplot.plot(x_loss, losses)

    if show:
        pyplot.show()

    pyplot.close()


## Define Model Architecture
- 33 inputs = 3 joint angles per leg, 4 legs, 2 DOF per joint. 4 touch sensors. 1 sine timestamp.
- 28 outputs = *same as above, except just the joint angles*


In [61]:
class GaitModel(nn.Module):
    def __init__(self, layer_sizes):
        super(GaitModel, self).__init__()


        hidden_layers = [
            nn.Sequential(nn.Linear(nlminus1, nl), nn.ReLU(), nn.BatchNorm1d(nl))
            for nl, nlminus1 in zip(layer_sizes[1:-1], layer_sizes)
        ]

        # The output layer does not include an activation function.
        # See: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
        output_layer = nn.Linear(layer_sizes[-2], layer_sizes[-1])

        tanh = torch.nn.Tanh()
        
        # Group all layers into the sequential container
        all_layers = hidden_layers + [output_layer] + [tanh]
        self.layers = nn.Sequential(*all_layers)

    def forward(self, X):
        return self.layers(X)

In [62]:
# now deprecated in favor of variable length NN (above)
class PytorchBrain(torch.nn.Module):
    _id = 0 # Global genome identifier
    _num_inputs = 33
    _num_outputs = 28

    # @classmethod
    # def __get_new_id(cls):
    #     cls._id += 1
    #     return cls._id

    @classmethod
    def get_num_outputs(cls):
        return cls._num_outputs

    def __init__(self):
        super().__init__()
        
        # Inputs to hidden layer linear transformation
        self.h1 = torch.nn.Linear(PytorchBrain._num_inputs, 12)
        self.relu = torch.nn.ReLU() #output is same shape as input
        self.batch_norm = torch.nn.BatchNorm1d(12)
        self.h2 = torch.nn.Linear(12, 12)
        self.h3 = torch.nn.Linear(12, 12)

        # Output layer
        self.output = torch.nn.Linear(12, PytorchBrain._num_outputs)
        
        # Define sigmoid activation
        self.sigmoid = torch.nn.Sigmoid()
        self.tanh = torch.nn.Tanh()
        
    def forward(self, x):
        # Pass the input tensor through each of our operations
        # TODO: collapse all x = into 
        # TODO: either sigmoid OR Relu
        x = self.h1(x)
        x = self.relu(x)
        x = self.batch_norm(x)
        # x = self.sigmoid(x)

        x = self.h2(x)
        x = self.relu(x)
        x = self.batch_norm(x)
        # x = self.sigmoid(x)

        x = self.h3(x)
        x = self.relu(x)
        x = self.batch_norm(x)
        # x = self.sigmoid(x)

        # TODO: linear then tanh might be better?
        x = self.tanh(x) #caps output -1 to 1
        x = self.output(x)
        
        return x

## Define Run function

In [63]:
def run(train_dataset, test_dataset, epochs=4, layer_sizes=[33, 31, 30, 28]):
    # Batch size is the number of training examples used to calculate each iteration's gradient
    batch_size_train = 33
    
    data_loader_train = DataLoader(dataset=train_dataset, batch_size=batch_size_train, shuffle=True)
    data_loader_test = DataLoader(dataset=test_dataset, batch_size=len(test_dataset), shuffle=False)
    
    # Define the hyperparameters
    learning_rate = 1e-3

    pytorch_model = GaitModel(layer_sizes)
    
    # Initialize the optimizer with above parameters
    optimizer = optim.Adam(pytorch_model.parameters(), lr=learning_rate)

    # Define the loss function
    loss_fn = nn.MSELoss()  # mean squared error

    # Train and get the resulting loss per iteration
    loss = train(model=pytorch_model, loader=data_loader_train, optimizer=optimizer, loss_fn=loss_fn, epochs=epochs)
    

    # Test and get the resulting predicted y values
    y_predict = test(model=pytorch_model, loader=data_loader_test)

    return loss, y_predict, pytorch_model

## Run and plot results

In [64]:

ic(train_dataset.length)
ic(test_dataset.length)

losses, y_predict, model_to_save = run(train_dataset=train_dataset, test_dataset=test_dataset, epochs=400)
torch.save(model_to_save, 'model.pt')

ic("Final loss:", sum(losses[-100:])/100)
plot_loss(losses)

# Transpose the matrices so we can plot.
# y_test = test_dataset.y_data.transpose()
# y_predict = y_predict.transpose()
y_test = np.transpose(test_dataset.y_data)
y_predict = np.transpose(y_predict)


# for yt, yp in zip(y_test, y_predict):
#   fig2 = pyplot.figure(dpi=300)
#   fig2.set_size_inches(8,6)
#   ic(test_dataset.x_data.shape)
#   ic(yt.shape)
#   ic(yp.shape)
#   pyplot.scatter(test_dataset.x_data, yt, marker='o', s=0.2)
#   ic('second scatter')
#   pyplot.scatter(test_dataset.x_data, yp, marker='o', s=0.3)
#   pyplot.text(-9, 0.44, "- Prediction", color="orange", fontsize=8)
#   pyplot.text(-9, 0.48, "- Sine (with noise)", color="blue", fontsize=8)
#   pyplot.show()


ic| train_dataset.length: 75
ic| test_dataset.length: 19
F:\Anaconda\envs\datascience\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([33, 24])) that is different to the input size (torch.Size([33, 28])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (28) must match the size of tensor b (24) at non-singleton dimension 1

In [9]:
losses

[2.3299989700317383,
 2.2911760807037354,
 2.493915557861328,
 2.4274396896362305,
 2.3643479347229004,
 2.3504559993743896,
 2.2515764236450195,
 2.369004249572754,
 2.2836692333221436,
 2.208125114440918,
 2.2971057891845703,
 2.2098734378814697,
 2.24356746673584,
 2.0889909267425537,
 2.168239116668701,
 2.149796962738037,
 2.2388055324554443,
 2.130577325820923,
 2.028994560241699,
 2.1951258182525635,
 2.1261837482452393,
 2.0404958724975586,
 2.0034430027008057,
 2.052830934524536,
 2.0382909774780273,
 1.993867039680481,
 2.049422264099121,
 1.9392725229263306,
 1.919783115386963,
 1.9625155925750732,
 1.8850502967834473,
 1.970686435699463,
 1.8677446842193604,
 1.9322062730789185,
 1.9262807369232178,
 1.7983603477478027,
 1.7795259952545166,
 1.8274887800216675,
 1.8282990455627441,
 1.780476450920105,
 1.9193055629730225,
 1.7257955074310303,
 1.8326306343078613,
 1.7721259593963623,
 1.6594632863998413,
 1.662449836730957,
 1.7343165874481201,
 1.7193888425827026,
 1.74601